# PFDA Big Project (working title)
## Author: Andre Hoarau

* Country: The name of the country where the health data was recorded.
* Year: The year in which the data was collected.
* Disease Name: The name of the disease or health condition tracked.
* Disease Category: The category of the disease (e.g., Infectious, Non-Communicable).
* Prevalence Rate (%): The percentage of the population affected by the disease.
* Incidence Rate (%): The percentage of new or newly diagnosed cases.
* Mortality Rate (%): The percentage of the affected population that dies from the disease.
* Age Group: The age range most affected by the disease.
* Gender: The gender(s) affected by the disease (Male, Female, Both).
* Population Affected: The total number of individuals affected by the disease.
* Healthcare Access (%): The percentage of the population with access to healthcare.
* Doctors per 1000: The number of doctors per 1000 people.
* Hospital Beds per 1000: The number of hospital beds available per 1000 people.
* Treatment Type: The primary treatment method for the disease (e.g., Medication, Surgery).
* Average Treatment Cost (USD): The average cost of treating the disease in USD.
* Availability of Vaccines/Treatment: Whether vaccines or treatments are available.
* Recovery Rate (%): The percentage of people who recover from the disease.
* DALYs: Disability-Adjusted Life Years, a measure of disease burden.
* Improvement in 5 Years (%): The improvement in disease outcomes over the last five years.
* Per Capita Income (USD): The average income per person in the country.
* Education Index: The average level of education in the country.
* Urbanization Rate (%): The percentage of the population living in urban areas.

In [2]:
# Imports that we will need
import pandas as pd
import plotly.express as px
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt


In [7]:
# Read in our first data set
filepath = "../pfda_projectdata/updatedglobalhealthstatistics.zip"

dfglobalhealth = pd.read_csv(filepath, compression='zip' )
dfglobalhealth.head()


,Country,Year,Disease Name,Disease Category,Prevalence Rate (%),Incidence Rate (%),Mortality Rate (%),Age Group,Gender,Population Affected,...,Hospital Beds per 1000,Treatment Type,Average Treatment Cost (USD),Availability of Vaccines/Treatment,Recovery Rate (%),DALYs,Improvement in 5 Years (%),Per Capita Income (USD),Education Index,Urbanization Rate (%)
0,Italy,2013,Malaria,Respiratory,0.95,1.55,8.42,0-18,Male,471007,...,7.58,Medication,21064,No,91.82,4493,2.16,16886,0.79,86.02
1,France,2002,Ebola,Parasitic,12.46,8.63,8.75,61+,Male,634318,...,5.11,Surgery,47851,Yes,76.65,2366,4.82,80639,0.74,45.52
2,Turkey,2015,COVID-19,Genetic,0.91,2.35,6.22,36-60,Male,154878,...,3.49,Vaccination,27834,Yes,98.55,41,5.81,12245,0.41,40.20
3,Indonesia,2011,Parkinson's Disease,Autoimmune,4.68,6.29,3.99,0-18,Other,446224,...,8.44,Surgery,144,Yes,67.35,3201,2.22,49336,0.49,58.47
4,Italy,2013,Tuberculosis,Genetic,0.83,13.59,7.01,61+,Male,472908,...,5.90,Medication,8908,Yes,50.06,2832,6.93,47701,0.50,48.14


In [8]:
# Create a database file for efficient use of the data.
db_path = "../pfda_projectdata/my_database.db"  # Path to your SQLite database file
conn = sqlite3.connect(db_path)
table_name = "country_health_data"
dfglobalhealth.to_sql(table_name, conn, if_exists="replace", index=False)


conn.close

<function Connection.close()>

In [9]:
conn = sqlite3.connect(db_path)

query = """
SELECT *
FROM country_health_data
WHERE "Mortality Rate (%)" IS NOT NULL;
""" 
dfglobalhealth = pd.read_sql(query, conn)

# Deadliest disease recorded per country
deadliestdisease = dfglobalhealth.loc[dfglobalhealth.groupby("Country")["Mortality Rate (%)"].idxmax()]
# Create a choropleth map
fig = px.choropleth(
    deadliestdisease,
    locations="Country",             # Use country names directly
    locationmode="country names",    # Specify that we're using country names
    color="Mortality Rate (%)",          # Values to color by
    hover_name="Country",            # Display country name on hover
    hover_data={"Disease Name": True, "Mortality Rate (%)": True},  # Additional data on hover
    title="Deadliest Disease by Country",
    color_continuous_scale="Reds"    # Color scale for the map
)

fig.update_layout(geo=dict(showframe=False, showcoastlines=True))
fig.show()
conn.close()

### Noticing that this only had 20 countries I expanded my search and used the Institute for Health Metrics and Evaluation (IHME) to gather global mortality by infectious diseases.

conn = sqlite3.connect(db_path)
table_name = "world_mortality_data"
dfglobalhealth.to_sql(table_name, conn, if_exists="replace", index=False)


conn.close

In [17]:
# Read in our second data set
filepath2 = "../pfda_projectdata/ihmemortalitydata.zip"
mortalitydata = pd.read_csv(filepath2, compression='zip' )
mortalitydata.head()

,measure,location,sex,age,cause,metric,year,val,upper,lower
0,Deaths,China,Both,All ages,Respiratory infections and tuberculosis,Number,2021,247527.122941,298829.846631,205348.321356
1,Deaths,China,Both,All ages,Respiratory infections and tuberculosis,Percent,2021,0.021173,0.024804,0.018712
2,Deaths,China,Both,All ages,Respiratory infections and tuberculosis,Rate,2021,17.397844,21.003739,14.433239
3,Deaths,China,Both,All ages,Enteric infections,Number,2021,5593.531658,9187.547949,3929.904519
4,Deaths,China,Both,All ages,Enteric infections,Percent,2021,0.000478,0.000741,0.000337


# References:
[MalaiarasuGRaj. (2024). Global Health Statistics [Data set]. Kaggle.](https://doi.org/10.34740/KAGGLE/DSV/10028650) - This is the global health statistics I used.

# End